In [1]:
import geopandas as gpd
import matplotlib.pyplot as plt
import mplleaflet
import pandas as pd
import base64
from folium import IFrame
import folium


In [2]:
gdf = gpd.read_parquet("3day_s.parquet")
gdf

,sd,geometry
time,,
2022-12-29 12:00:00,10.000000,POINT (65.200 -76.200)
2022-12-29 12:00:00,0.129248,POINT (97.500 64.700)
2022-12-29 12:00:00,0.000000,POINT (82.900 22.900)
2022-12-29 12:00:00,0.000000,POINT (27.900 -8.200)
2022-12-29 12:00:00,0.084843,POINT (102.400 69.800)
...,...,...
2022-12-31 12:00:00,0.001831,POINT (117.800 29.800)
2022-12-31 12:00:00,0.000000,POINT (98.000 27.300)
2022-12-31 12:00:00,0.074008,POINT (34.900 56.700)


#### Online example (https://livecodestream.dev/post/how-to-plot-your-data-on-maps-using-python-and-folium/)

In [27]:
import json

import requests

url = (
    "https://raw.githubusercontent.com/python-visualization/folium/master/examples/data"
)
data = json.loads(requests.get(f"{url}/vis1.json").text)

data

{'axes': [{'scale': 'x', 'title': 'Time', 'type': 'x'},
  {'scale': 'y', 'title': 'Dominant Wave Period (s)', 'type': 'y'}],
 'data': [{'name': 'table',
   'values': [{'col': 'dominant_wave_period (s)',
     'idx': 1366257000000,
     'val': 8.0},
    {'col': 'dominant_wave_period (s)', 'idx': 1366260600000, 'val': 6.0},
    {'col': 'dominant_wave_period (s)', 'idx': 1366264200000, 'val': 7.0},
    {'col': 'dominant_wave_period (s)', 'idx': 1366267800000, 'val': 6.0},
    {'col': 'dominant_wave_period (s)', 'idx': 1366271400000, 'val': 8.0},
    {'col': 'dominant_wave_period (s)', 'idx': 1366275000000, 'val': 6.0},
    {'col': 'dominant_wave_period (s)', 'idx': 1366278600000, 'val': 6.0},
    {'col': 'dominant_wave_period (s)', 'idx': 1366282200000, 'val': 6.0},
    {'col': 'dominant_wave_period (s)', 'idx': 1366285800000, 'val': 8.0},
    {'col': 'dominant_wave_period (s)', 'idx': 1366289400000, 'val': 7.0},
    {'col': 'dominant_wave_period (s)', 'idx': 1366293000000, 'val': 7.0},
  

In [28]:
m = folium.Map(location=[48.218871184761596, 11.624819877497147], zoom_start=15, tiles="Stamen Terrain")

marker = folium.Marker(
    location=[48.218871184761596, 11.624819877497147],
    popup=folium.Popup(max_width=450).add_child(
        folium.Vega(data, width=450, height=250)
    ),
)

marker.add_to(m)

m

#### Prototype example w/ my data

In [1]:
import geopandas as gpd
import matplotlib.pyplot as plt
import mplleaflet
import pandas as pd
import base64
from folium import IFrame
import folium

In [2]:
gdf = gpd.read_parquet("full_data.parquet")

In [29]:

from shapely import Point
import altair

# Good snowfall spot
event = Point((48.7767, 121.8144))

clicked_point = Point(event.x, event.y)

point_dist = gdf.distance(clicked_point)
closest_points = gdf[point_dist==point_dist.min()]

closest_points


,sd,time,geometry
320906,0.127569,2022-12-29 12:00:00,POINT (50.60000 81.10000)
6804506,0.128180,2022-12-30 12:00:00,POINT (50.60000 81.10000)
13288106,0.130774,2022-12-31 12:00:00,POINT (50.60000 81.10000)


#### Looked up Vega visualization. Folium docs said altair support (https://python-visualization.github.io/folium/modules.html)
#### ChatGPT helped with altair chart

In [31]:
#y_axis_bounds = altair.Scale(domain=[closest_points.sd.max(), 0])

width = 300
height = 200


vega_data = altair.Chart(closest_points, width=width, height=height).mark_line().encode(
    x = 'time:T',
    y = 'sd:N'
)
vega_data

alt.Chart(...)

In [32]:
lat = event.x
lon = event.y

m = folium.Map(location=[lat, lon], zoom_start=15)

marker = folium.Marker(
    location=[lat, lon],
    popup=folium.Popup(max_width=450).add_child(
        folium.VegaLite(vega_data, width=450, height=250)
    ),
)

marker.add_to(m)

m

## Need mouse position from click

#### Get mouse position from click

In [ ]:
import folium
from folium.plugins import MousePosition


m = folium.Map()

MousePosition().add_to(m)

m

this is just a lat/lng display

#### Look at click for marker

In [26]:

lat = event.x
lon = event.y

m = folium.Map(location=[lat, lon], zoom_start=15)

m.add_child(
    folium.ClickForMarker(
        popup=folium.Popup(max_width=450).add_child(
            folium.VegaLite(vega_dict, width=450, height=250)
        )
    )
)

m

AssertionError: You cannot render this Element if it is not in a Figure.

#### Internet says need streamlit for vega onclick lat/lon

#### Prototype time plot from click point

In [23]:
from shapely import Point

event = Point((12, 18))


clicked_point = Point(event.x, event.y)

point_dist = gdf.distance(clicked_point)
closest_points = point_dist[point_dist==point_dist.min()]

closest_points


time
2022-12-29 12:00:00    0.1
dtype: float64

In [ ]:
# Get all points nearest to click
#nearest_row = gdf.distance(clicked_point).idxmin()
nearest_point_idx = gdf.distance(clicked_point).min()
#nearest_point = gdf["geometry"].iloc[nearest_point_idx]
#selected_data = gdf[gdf["geometry"]==nearest_point]

#vega_data = to_vega(selected_data)


nearest_point_idx